In [2]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [5]:
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

In [6]:
env = gym_super_mario_bros.make("SuperMarioBros-v0")
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order="last")

In [7]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [8]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, "best_model_{}".format(self.n_calls))
            self.model.save(model_path)
        return True

In [9]:
CHECKPOINT_DIR = "./train/"
LOG_DIR = "./logs/"

In [10]:
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [19]:
model = PPO("CnnPolicy", env, device="cuda", verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.001, n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [20]:
model.learn(total_timesteps=1000, callback=callback)

Logging to ./logs/PPO_3
----------------------------
| time/              |     |
|    fps             | 70  |
|    iterations      | 1   |
|    time_elapsed    | 7   |
|    total_timesteps | 512 |
----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 13         |
|    iterations           | 2          |
|    time_elapsed         | 78         |
|    total_timesteps      | 1024       |
| train/                  |            |
|    approx_kl            | 0.30583236 |
|    clip_fraction        | 0.653      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.82      |
|    explained_variance   | 0.00685    |
|    learning_rate        | 0.001      |
|    loss                 | 3.83       |
|    n_updates            | 10         |
|    policy_gradient_loss | 0.0989     |
|    value_loss           | 147        |
----------------------------------------


KeyboardInterrupt: 

In [ ]:
state = env.reset()
while True: 
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()